In [1]:
import h5py
import json
import torch
import os
import gradio as gr
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import subprocess

In [2]:
import shutil

In [3]:
shutil.rmtree("/home/vivek.trivedi/ET623_project/")

In [2]:
!pip uninstall param -y

In [3]:
# path="/scratch/vivek.trivedi/VL_adapter/datasets/COCO/dataset_coco.json"
# with open(path, 'r') as json_file:
#     karpathy_data = json.load(json_file)

In [4]:
# n_images = 0
# data = []
# for datum in karpathy_data['images']:
#         for d in datum['sentences']:
#             #img_id = str(int(datum['filename'].split('.')[0].split('_')[-1]))
# #             else:
#             img_id = datum['filename'].split('.')[0]
#             new_datum = {
#                 'img_id': img_id,
#                 'sent': d['raw'].strip(),
#                 'targets': [d['raw'].strip() for d in datum['sentences']],
#                 'is_train': False,
#             }
#             data.append(new_datum)
#         n_images += 1

In [5]:
cd

/home/vivek.trivedi


In [6]:
cd "/scratch/vivek.trivedi/VL_adapter/VL-T5/src"

/scratch/vivek.trivedi/VL_adapter/VL-T5/src


In [7]:
from param import parse_args

In [8]:
args = parse_args(
    parse=False,
    backbone='facebook/bart-base',
    load="/scratch/vivek.trivedi/VL_adapter/VL-T5/snap/VLBart_multitask/4tasks_hard_RN101_LMfull_bs100_image224_lr1e-4/LAST"
)
args.gpu = 1

In [9]:
trainer_dic={"Full finetune":"/scratch/vivek.trivedi/VL_adapter/VL-T5/snap/VLBart_multitask/4tasks_hard_RN101_LMfull_bs100_image224_lr1e-4/LAST",
             "Single Adapter":"/scratch/vivek.trivedi/VL_adapter/VL-T5/snap/VLBart_multitask/4tasks_hard_RN101_LMOneadapter+r8+ln_bs100_image224_lr1e-3/LAST",
             "Multi Adapter":"/scratch/vivek.trivedi/VL_adapter/VL-T5/snap/VLBart_multitask/4tasks_hard_RN101_LMadapter+r8+ln_bs80_image224_lr3e-4/LAST",
             "Hyperformer":"/scratch/vivek.trivedi/VL_adapter/VL-T5/snap/VLBart_multitask/4tasks_hard_RN101_LMhyperformer8+r8+ln_bs100_image224_lr1e-3/LAST",
             "Single Compacter":"/scratch/vivek.trivedi/VL_adapter/VL-T5/snap/VLBart_multitask/4tasks_hard_RN101_LMOnecompacter+hdiv2+noshare+nofac+ln+prompt_bs100_image224_lr1e-3/LAST",
             "Multi Compacter":"/scratch/vivek.trivedi/VL_adapter/VL-T5/snap/VLBart_multitask/4tasks_hard_RN101_LMcompacter+hdiv2+noshare+nofac+ln+prompt_bs100_image224_lr1e-3/LAST",
             "Single LoRa":"/scratch/vivek.trivedi/VL_adapter/VL-T5/snap/VLBart_multitask/RN101_LMsinglelora128+lr1e-3_bs100_image224/LAST",
             "Multi Lora":"/scratch/vivek.trivedi/VL_adapter/VL-T5/snap/VLBart_multitask/RN101_LMmultilora128+lr1e-3_bs100_image224/LAST",
             "Prompt Tunning":"/scratch/vivek.trivedi/VL_adapter/VL-T5/snap/VLBart_multitask/4tasks_hard_RN101_LMprompt40_bs70_image224_lr1e-3/LAST"
            }

In [10]:
from multitask import Trainer

In [11]:
trainer = Trainer(args,
                  train=False
                  )

Building Model at GPU 1
**************************************************
Configurations
{'RefCOCO_BUTD': False,
 'RefCOCO_GT': False,
 'adam_beta1': 0.9,
 'adam_beta2': 0.999,
 'adam_eps': 1e-06,
 'add_adapter_cross_attn': True,
 'add_layer_norm_after_adapter': False,
 'add_layer_norm_before_adapter': False,
 'additional_visual_embedding_layers': 0,
 'answer_normalize': False,
 'backbone': 'facebook/bart-base',
 'batch_size': 256,
 'caption_cocoonly': True,
 'caption_only': False,
 'classifier': False,
 'clip_grad_norm': -1.0,
 'cls_task': 'tinyimagenet',
 'coco_only': False,
 'comment': '',
 'decoder_prompt_len': 0,
 'deepspeed': None,
 'distributed': False,
 'do_lower_case': False,
 'downsample': False,
 'dropout': 0.1,
 'dry': False,
 'efficient_unique_hyper_net': False,
 'encoder_prompt_len': 0,
 'epochs': 12,
 'expand_vis_embedding': False,
 'factorized_phm': True,
 'feat_dim': 2048,
 'feature_type': 'butd',
 'fp16': False,
 'freeze_bn_statistics': False,
 'freeze_ln_statistics'

In [12]:
from transformers import T5TokenizerFast, BartTokenizer
tokenizer = BartTokenizer.from_pretrained(
    args.backbone,
    # max_length=self.args.max_text_length,
    do_lower_case=True)

In [13]:
def caption_build(index):
    image_name=data[index]['img_id']
    path="/scratch/vivek.trivedi/VL_adapter/datasets/COCO/clip_features/data_clip_RN101_att/"+image_name+".h5"
    with h5py.File(path, 'r') as file:
        features = file[image_name]["features"][:]
        features=torch.tensor(features)
    input_text = ''
    input_ids = []
    input_ids =tokenizer.encode(input_text)
    B=1
    input_ids=torch.LongTensor([input_ids]).type(torch.long)
    boxes = torch.zeros(B,features.shape[0], 4,dtype=torch.float)
    features=torch.unsqueeze(features, dim=0).type(torch.float)
    batch={}
    batch['vis_feats'] = features # (L, D)
    batch['boxes'] = boxes
    batch['task']="caption"
    batch["input_ids"]=input_ids
    result = trainer.model.test_step(batch)
    print("prediction of model is ")
    print(result['pred'])
    if 'val2014' in image_name:
        image_path="/scratch/vivek.trivedi/VL_adapter/datasetsp/COCO/images/val2014/"+image_name+".jpg"
    elif 'train2014' in image_name:
        image_path="/scratch/vivek.trivedi/VL_adapter/datasetsp/COCO/images/train2014/"+image_name+".jpg"
    elif 'test2015' in image_name:
        image_path="/scratch/vivek.trivedi/VL_adapter/datasetsp/COCO/images/test2015/"+image_name+".jpg"
    else:
        None
    img = mpimg.imread(image_path)
    plt.imshow(img)
    plt.axis('off')
    plt.show()  

In [14]:
# caption_build(1000)

In [15]:
def run_clip_visual_feats(image_name):
    os.chdir('/home/vivek.trivedi/OpenAI-CLIP-Feature/')
    command = f"""\
    CUDA_VISIBLE_DEVICES=1 python3 clip_visual_feats.py \
    --image_list {image_name} \
    --image_dir "/scratch/vivek.trivedi/VL_adapter/vivek_exp/uploaded_image/" \
    --output_dir "/scratch/vivek.trivedi/VL_adapter/vivek_exp/features/" \
    --ve_name 'RN101' \
    --model_type_or_path 'RN101'
    """
    try:
        # Run the command within the current Python environment
        subprocess.run(command, shell=True, check=True, executable="/bin/bash")
        print("Command executed successfully.")
    except subprocess.CalledProcessError as e:
        print("Error: ", e)
    os.chdir('/home/vivek.trivedi/')

In [16]:
cd

/home/vivek.trivedi


In [17]:
cd "/scratch/vivek.trivedi/VL_adapter/VL-T5/src"

/scratch/vivek.trivedi/VL_adapter/VL-T5/src


In [18]:
trainer_dict={}
for key,value in trainer_dic.items():
    args = parse_args(
        parse=False,
        backbone='facebook/bart-base',
        load=value
    )
    args.gpu = 1
    trainer_ = None
    trainer_dict[key]=trainer_

In [19]:
def caption_build(image_name,task,question):
    run_clip_visual_feats(image_name)
    image_path="/scratch/vivek.trivedi/VL_adapter/vivek_exp/uploaded_image/"+image_name
    features=torch.tensor(np.load("/scratch/vivek.trivedi/VL_adapter/vivek_exp/features/"+image_name+".npz")['features'])
    input_text = question
    input_ids = []
    input_ids =tokenizer.encode(input_text)
    B=1
    input_ids=torch.LongTensor([input_ids]).type(torch.long)
    boxes = torch.zeros(B,features.shape[0], 4,dtype=torch.float)
    features=torch.unsqueeze(features, dim=0).type(torch.float)
    batch={}
    batch['vis_feats'] = features # (L, D)
    batch['boxes'] = boxes
    batch['task']=task
    batch["input_ids"]=input_ids
    result = trainer.model.test_step(batch)
    print("prediction of model is ")
    print(result)
    img = mpimg.imread(image_path)
    plt.imshow(img)
    plt.axis('off')
    plt.show()  

In [20]:
def vqa_pred(image,question,trainer_adapter):
    list_vq=list(trainer_dic.keys())
    if trainer_adapter!=None:
        trainer_ = trainer_dict[list_vq[trainer_adapter]]
    image_name="image1.jpg"
    image.save("/scratch/vivek.trivedi/VL_adapter/vivek_exp/uploaded_image/" + image_name)
    run_clip_visual_feats(image_name)
    image_path="/scratch/vivek.trivedi/VL_adapter/vivek_exp/uploaded_image/"+image_name
    features=torch.tensor(np.load("/scratch/vivek.trivedi/VL_adapter/vivek_exp/features/"+image_name+".npz")['features'])
    input_text = question
    input_ids = []
    input_ids =tokenizer.encode(input_text)
    B=1
    input_ids=torch.LongTensor([input_ids]).type(torch.long)
    boxes = torch.zeros(B,features.shape[0], 4,dtype=torch.float)
    features=torch.unsqueeze(features, dim=0).type(torch.float)
    batch={}
    batch['vis_feats'] = features # (L, D)
    batch['boxes'] = boxes
    batch['task']='vqa'
    batch["input_ids"]=input_ids
    result = trainer.model.test_step(batch)
    if os.path.exists("/scratch/vivek.trivedi/VL_adapter/vivek_exp/uploaded_image/" + image_name):
        os.remove("/scratch/vivek.trivedi/VL_adapter/vivek_exp/uploaded_image/" + image_name)
    if os.path.exists("/scratch/vivek.trivedi/VL_adapter/vivek_exp/features/" + image_name + ".npz"):
        os.remove("/scratch/vivek.trivedi/VL_adapter/vivek_exp/features/" + image_name + ".npz")
    return result["pred_ans"][0]

In [21]:
def cap_pred(image,trainer_adapter):
    list_vq=list(trainer_dic.keys())
    if trainer_adapter!=None:
        trainer_ = trainer_dict[list_vq[trainer_adapter]]
    question=""
    image_name="image1.jpg"
    image.save("/scratch/vivek.trivedi/VL_adapter/vivek_exp/uploaded_image/" + image_name)
    run_clip_visual_feats(image_name)
    image_path="/scratch/vivek.trivedi/VL_adapter/vivek_exp/uploaded_image/"+image_name
    features=torch.tensor(np.load("/scratch/vivek.trivedi/VL_adapter/vivek_exp/features/"+image_name+".npz")['features'])
    input_text = question
    input_ids = []
    input_ids =tokenizer.encode(input_text)
    B=1
    input_ids=torch.LongTensor([input_ids]).type(torch.long)
    boxes = torch.zeros(B,features.shape[0], 4,dtype=torch.float)
    features=torch.unsqueeze(features, dim=0).type(torch.float)
    batch={}
    batch['vis_feats'] = features # (L, D)
    batch['boxes'] = boxes
    batch['task']='caption'
    batch["input_ids"]=input_ids
    result = trainer.model.test_step(batch)
    if os.path.exists("/scratch/vivek.trivedi/VL_adapter/vivek_exp/uploaded_image/" + image_name):
        os.remove("/scratch/vivek.trivedi/VL_adapter/vivek_exp/uploaded_image/" + image_name)
    if os.path.exists("/scratch/vivek.trivedi/VL_adapter/vivek_exp/features/" + image_name + ".npz"):
        os.remove("/scratch/vivek.trivedi/VL_adapter/vivek_exp/features/" + image_name + ".npz")
    return result["pred"]

In [22]:
def gqa_pred(image,question,trainer_adapter):
    list_vq=list(trainer_dic.keys())
    if trainer_adapter!=None:
        trainer_ = trainer_dict[list_vq[trainer_adapter]]
    image_name="image1.jpg"
    image.save("/scratch/vivek.trivedi/VL_adapter/vivek_exp/uploaded_image/" + image_name)
    run_clip_visual_feats(image_name)
    image_path="/scratch/vivek.trivedi/VL_adapter/vivek_exp/uploaded_image/"+image_name
    features=torch.tensor(np.load("/scratch/vivek.trivedi/VL_adapter/vivek_exp/features/"+image_name+".npz")['features'])
    input_text = question
    input_ids = []
    input_ids =tokenizer.encode(input_text)
    B=1
    input_ids=torch.LongTensor([input_ids]).type(torch.long)
    boxes = torch.zeros(B,features.shape[0], 4,dtype=torch.float)
    features=torch.unsqueeze(features, dim=0).type(torch.float)
    batch={}
    batch['vis_feats'] = features # (L, D)
    batch['boxes'] = boxes
    batch['task']='gqa'
    batch["input_ids"]=input_ids
    result = trainer.model.test_step(batch)
    if os.path.exists("/scratch/vivek.trivedi/VL_adapter/vivek_exp/uploaded_image/" + image_name):
        os.remove("/scratch/vivek.trivedi/VL_adapter/vivek_exp/uploaded_image/" + image_name)
    if os.path.exists("/scratch/vivek.trivedi/VL_adapter/vivek_exp/features/" + image_name + ".npz"):
        os.remove("/scratch/vivek.trivedi/VL_adapter/vivek_exp/features/" + image_name + ".npz")
    return result["pred_ans"][0]

In [23]:
def nlvr_pred(image1, image2, question,trainer_adapter):
    list_vq=list(trainer_dic.keys())
    if trainer_adapter!=None:
        trainer_ = trainer_dict[list_vq[trainer_adapter]]
    image_name1="image1.jpg"
    image_name2="image2.jpg"
    image1.save("/scratch/vivek.trivedi/VL_adapter/vivek_exp/uploaded_image/" + image_name1)
    image2.save("/scratch/vivek.trivedi/VL_adapter/vivek_exp/uploaded_image/" + image_name2)
    run_clip_visual_feats(image_name1)
    image_path1 = "/scratch/vivek.trivedi/VL_adapter/vivek_exp/uploaded_image/" + image_name1
    features1 = torch.tensor(np.load("/scratch/vivek.trivedi/VL_adapter/vivek_exp/features/" + image_name1 + ".npz")['features'])

    run_clip_visual_feats(image_name2)
    image_path2 = "/scratch/vivek.trivedi/VL_adapter/vivek_exp/uploaded_image/" + image_name2
    features2 = torch.tensor(np.load("/scratch/vivek.trivedi/VL_adapter/vivek_exp/features/" + image_name2 + ".npz")['features'])

    input_text = question
    input_ids = tokenizer.encode(input_text)
    input_ids = torch.LongTensor([input_ids]).type(torch.long)

    B = 1
    features1 = torch.unsqueeze(features1, dim=0).unsqueeze(0).type(torch.float)
    features2 = torch.unsqueeze(features2, dim=0).unsqueeze(0).type(torch.float)

    # Concatenate features1 and features2 along a new dimension 1
    features = torch.cat((features1, features2), dim=1)
    boxes1 = torch.zeros(B, features1.shape[2], 4, dtype=torch.float).unsqueeze(0)
    boxes2 = torch.zeros(B, features2.shape[2], 4, dtype=torch.float).unsqueeze(0)

    # Concatenate boxes1 and boxes2 along a new dimension 1
    boxes = torch.cat((boxes1, boxes2), dim=1)
    batch = {}
    batch['vis_feats'] = features  # (L, D)
    batch['boxes'] = boxes
    batch['task'] = 'nlvr'
    batch["input_ids"] = input_ids

    result = trainer.model.test_step(batch)
    if os.path.exists("/scratch/vivek.trivedi/VL_adapter/vivek_exp/uploaded_image/" + image_name1):
        os.remove("/scratch/vivek.trivedi/VL_adapter/vivek_exp/uploaded_image/" + image_name1)
    if os.path.exists("/scratch/vivek.trivedi/VL_adapter/vivek_exp/features/" + image_name1 + ".npz"):
        os.remove("/scratch/vivek.trivedi/VL_adapter/vivek_exp/features/" + image_name1 + ".npz")
    if os.path.exists("/scratch/vivek.trivedi/VL_adapter/vivek_exp/uploaded_image/" + image_name2):
        os.remove("/scratch/vivek.trivedi/VL_adapter/vivek_exp/uploaded_image/" + image_name2)
    if os.path.exists("/scratch/vivek.trivedi/VL_adapter/vivek_exp/features/" + image_name2 + ".npz"):
        os.remove("/scratch/vivek.trivedi/VL_adapter/vivek_exp/features/" + image_name2 + ".npz")
    if result['pred_ans_id'].any():
        result_string = "True"
    else:
        result_string = "False"
    return result_string

In [24]:
theme = gr.themes.Base(
    primary_hue="gray",
    secondary_hue="orange",
    neutral_hue="neutral",
    font=[gr.themes.GoogleFont('Source Sans Pro'), 'ui-sans-serif', gr.themes.GoogleFont('system-ui'), 'sans-serif'],
).set(
    body_background_fill='*primary_200',
    body_background_fill_dark='*primary_950',
    body_text_color_dark='*primary_100',
    body_text_weight='500',
    embed_radius='*radius_xxl',
    background_fill_primary='*primary_50',
    button_shadow='*input_shadow',
    background_fill_primary_dark='*secondary_800',
    background_fill_secondary_dark='*primary_200',
    border_color_accent='*secondary_800',
    border_color_accent_dark='*secondary_950',
    border_color_primary='*primary_950',
    color_accent_soft_dark='*neutral_500',
    link_text_color='*neutral_900',
    link_text_color_dark='*primary_200',
    code_background_fill='*secondary_200',
    block_background_fill='*body_background_fill',
    button_border_width='*block_label_border_width',
    button_primary_text_color_dark='*primary_300',
    button_secondary_text_color_dark='*table_odd_background_fill',
    button_secondary_text_color_hover='*button_secondary_background_fill'
)

In [25]:
def demo():
    list_vq=list(trainer_dic.keys())
    with gr.Blocks(theme=theme) as demo:
        
        gr.Markdown(
            """<center><h2>Parameter Efficient training for Multimodal Tasks</center></h2>
            <h3>Full finetuning</h3>
            """
        )
        
        with gr.Tab("NLVR"):
            with gr.Row():
                vq_btn = gr.Radio(list_vq, \
                    label="Adapter models", type="index", info="Choose your Adapter model")
            with gr.Row():
                im1_nlvr=gr.Image(type="pil", label="Upload an first image")
                im2_nlvr=gr.Image(type="pil", label="Upload an second image")
            with gr.Row():
                ques_nlvr=gr.Textbox(label="write question related to image")
            with gr.Row():
                ans_nlvr=gr.Textbox(label="Answer")
            nlvr_submit = gr.Button("Generate NLVR Result...")
        with gr.Tab("VQA"):
            with gr.Row():
                vq_btn = gr.Radio(list_vq, \
                    label="Adapter models", type="index", info="Choose your Adapter model")
            with gr.Row():
                im1_vqa=gr.Image(type="pil", label="Upload an image")
            with gr.Row():
                ques_vqa=gr.Textbox(label="write question related to image")
            with gr.Row():
                ans_vqa=gr.Textbox(label="Answer")
            vqa_submit = gr.Button("Generate VQA Result...")
        with gr.Tab("GQA"):
            with gr.Row():
                vq_btn = gr.Radio(list_vq, \
                    label="Adapter models", type="index", info="Choose your Adapter model")
            with gr.Row():
                im1_gqa=gr.Image(type="pil", label="Upload an image")
            with gr.Row():
                ques_gqa=gr.Textbox(label="write question related to image")
            with gr.Row():
                ans_gqa=gr.Textbox(label="Answer")
            gqa_submit = gr.Button("Generate GQA Result...")
        with gr.Tab("Caption"):
            with gr.Row():
                vq_btn = gr.Radio(list_vq, \
                    label="Adapter models", type="index", info="Choose your Adapter model")
            with gr.Row():
                im1_cap=gr.Image(type="pil", label="Upload an image")
            with gr.Row():
                ans_cap=gr.Textbox(label="Answer")
            cap_submit = gr.Button("Generate caption Result...")
        nlvr_submit.click(nlvr_pred,inputs=[im1_nlvr,im2_nlvr,ques_nlvr,vq_btn],outputs=ans_nlvr)
        vqa_submit.click(vqa_pred,inputs=[im1_vqa,ques_vqa,vq_btn],outputs=ans_vqa)
        gqa_submit.click(gqa_pred,inputs=[im1_gqa,ques_gqa,vq_btn],outputs=ans_gqa)
        cap_submit.click(cap_pred,inputs=[im1_cap,vq_btn],outputs=ans_cap)
    demo.queue().launch(server_name='0.0.0.0',server_port=7072,debug=True)

In [ ]:
demo()

Running on local URL:  http://0.0.0.0:7072

To create a public link, set `share=True` in `launch()`.


In [ ]:
# image_path=os.listdir("/scratch/vivek.trivedi/VL_adapter/vivek_exp/uploaded_image/")
# nlvr_pred(image_path[5],image_path[4],'nlvr',"there are many person")

In [ ]:
# def caption_build_u(image_name,task,question):
#     run_clip_visual_feats(image_name)
#     image_path="/scratch/vivek.trivedi/VL_adapter/vivek_exp/uploaded_image/"+image_name
#     features=torch.tensor(np.load("/scratch/vivek.trivedi/VL_adapter/vivek_exp/features/"+image_name+".npz")['features'])
#     input_text = question
#     input_ids = []
#     input_ids =tokenizer.encode(input_text)
#     B=1
#     input_ids=torch.LongTensor([input_ids]).type(torch.long)
#     boxes = torch.zeros(B,features.shape[0], 4,dtype=torch.float)
#     features=torch.unsqueeze(features, dim=0).type(torch.float)
#     batch={}
#     batch['vis_feats'] = features # (L, D)
#     batch['boxes'] = boxes
#     batch['task']=task
#     batch["input_ids"]=input_ids
#     result = trainer.model.test_step(batch)
#     print("prediction of model is ")
#     print(result)
#     img = mpimg.imread(image_path)
# #     plt.imshow(img)
# #     plt.axis('off')
# #     plt.show()
#     return result

In [ ]:
# from ipywidgets import interact, widgets, Output
# from IPython.display import display, clear_output
# import io
# from PIL import Image
# import cv2
# import numpy as np
# import matplotlib.pyplot as plt

# def captionbuild(image, img_name, datatype, question):
#     # Read and process the uploaded image
#     content = image[list(image.keys())[0]]['content']
#     img = Image.open(io.BytesIO(content))
#     cv2.imwrite("/scratch/vivek.trivedi/VL_adapter/vivek_exp/uploaded_image/" + img_name + ".jpg", np.array(img))
#     caption_build_u(img_name+".jpg",datatype,question)
#     plt.imshow(img)
#     plt.axis('off')
#     plt.title(img_name)
#     plt.show()        

# # Create file upload widget for the input image and input fields for image name, datatype, and question
# upload_image = widgets.FileUpload(description="Upload Image")
# input_img_name = widgets.Text(description="Image Name:")
# input_datatype = widgets.Text(description="Datatype:")
# input_question = widgets.Text(description="Question:")

# # Create a submit button
# submit_button = widgets.Button(description="Submit")

# # Create an output widget for displaying text output
# text_output = Output()

# # Function to be executed when the submit button is clicked
# def on_submit_button_clicked(b):
#     with text_output:
#         clear_output()  # Clear previous output
#         captionbuild(upload_image.value, input_img_name.value, input_datatype.value, input_question.value)

# # Attach the function to the button click event
# submit_button.on_click(on_submit_button_clicked)

# # Display widgets and button
# display(upload_image, input_img_name, input_datatype, input_question, submit_button, text_output)

In [ ]:
# !pip install flask

In [ ]:
# from flask import Flask, render_template, request, redirect, url_for
# import io
# from PIL import Image
# import cv2
# import numpy as np
# import matplotlib.pyplot as plt
# from IPython.display import display, HTML

# app = Flask(__name__)
# CUDA_VISIBLE_DEVICES=1

# @app.route('/')
# def index():
#     return render_template('index.html')

# @app.route('/process', methods=['POST'])
# def process():
#     image = request.files['upload_image']
#     img_name = request.form['input_img_name']
#     datatype = request.form['action']
#     question = request.form.get('input_question', '')
#     content = image.read()
#     img = Image.open(io.BytesIO(content))
#     max_size = (800, 600)
#     img.thumbnail(max_size)
#     # Save the resized image
# #     if img_name+".jpg" in os.listdir("static/uploaded_image/"):
# #         os.remove("static/uploaded_image/" + img_name + ".jpg")
# #     if img_name+".jpg" in os.listdir("/home/vivek.trivedi/VL_adapter/vivek_exp/uploaded_image/"):
# #         os.remove("/home/vivek.trivedi/VL_adapter/vivek_exp/uploaded_image/" + img_name + ".jpg")
# #         os.remove("/home/vivek.trivedi/VL_adapter/vivek_exp/features/"+img_name+".npz")
#     img.save("static/uploaded_image/" + img_name + ".jpg")
#     cv2.imwrite("/scratch/vivek.trivedi/VL_adapter/vivek_exp/uploaded_image/" + img_name + ".jpg", np.array(img))
#     caption=caption_build_u(img_name+".jpg",datatype,question)

#     return  render_template('index.html', img_name=img_name, caption=caption)

# # Create an HTML template string
# html_template = """
# <!DOCTYPE html>
# <html>
# <head>
#     <title>Vision Question Answering Adapter</title>
# </head>
# <body>
#     <h1>Image Captioning</h1>
#     <form action="/process" method="POST" enctype="multipart/form-data">
#         <label for="upload_image">Upload Image:</label>
#         <input type="file" name="upload_image" required><br>
        
#         <label for="input_img_name">Image Name:</label>
#         <input type="text" name="input_img_name" required><br>
        
#         <label for="action">Datatype:</label>
#         <select name="action" required>
#             <option value="caption">caption of image</option>
#             <option value="vqa">question answering 1</option>
#         </select><br>
        
#         <label for="input_question">Question:</label>
#         <input type="text" name="input_question"><br>

#         <input type="submit" value="Submit">
#     </form>
#     {% if img_name %}
#     <h2>Uploaded Image: {{ img_name }}</h2>
#     <img src="{{ url_for('static', filename='/uploaded_image/' + img_name +'.jpg') }}" alt="Uploaded Image">
#     {% endif %}

#     {% if caption %}
#     <h2>Caption:</h2>
#     <p>{{ caption }}</p>
#     {% endif %}
# </body>
# </html>
# """

# # Save the HTML template to a file
# with open('templates/index.html', 'w') as f:
#     f.write(html_template)

# # Run the Flask app inside the Jupyter Notebook
# if __name__ == '__main__':
#     app.run(host='0.0.0.0', port=5000)
